# Playing around with some Portfolio Management evaluation ideas

### Imports

In [74]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Common imports
import numpy as np
import pandas as pd
import scipy.stats as stats
import os

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"


# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

### Random seed

In [75]:
# to make this notebook's output stable across runs
np.random.seed(42)

## Constants for the remainder

In [104]:
D = 480
MU = 0.0005
STD = 0.005

## Start with a uniform random variable centered at MU

*Though results may vary depending on the seed!!!!*

In [103]:
x = np.random.rand(D,1)/2


0.2544834329234661

In [105]:
days = np.arange(1,D+1)
days = np.reshape(days, (D,1))

months = days // 30 + 1

x = np.random.rand(D,1)
x = (x - x.mean())/x.std() * STD + MU
x = np.concatenate((days, months, x), axis=1)

daily_outperforms = pd.DataFrame(
    {'day': x[:, 0], 
     'month': x[:, 1],
     'outperformance': x[:, 2]})
daily_outperforms.describe()

,day,month,outperformance
count,480.000000,480.000000,480.000000
mean,240.500000,8.533333,0.000500
std,138.708327,4.618079,0.005005
min,1.000000,1.000000,-0.008209
25%,120.750000,5.000000,-0.003936
50%,240.500000,9.000000,0.000731
75%,360.250000,13.000000,0.004745
max,480.000000,17.000000,0.009087


In [87]:
def get_mu_sigma(outperforms):
    x = outperforms["outperformance"]
    return [x.mean(), x.std()]

In [88]:
mu_daily, sigma_daily = get_mu_sigma(daily_outperforms)
mu_daily, sigma_daily

(0.048167410268229384, 0.14950987969774493)

In [92]:
daily_prob_value_add = stats.norm.sf(0, loc=mu_daily, scale=sigma_daily)
daily_prob_value_add

0.6263375693641647

In [97]:
do_gb = daily_outperforms.drop(labels=['day'], axis=1)
do_gb = do_gb.groupby(['month'])
monthly_outperforms = do_gb.aggregate(np.mean)
monthly_outperforms.describe()

,outperformance
count,17.000000
mean,0.048885
std,0.021966
min,0.007067
25%,0.038632
50%,0.046948
75%,0.052168
max,0.100428


In [98]:
mu_monthly, sigma_monthly = get_mu_sigma(monthly_outperforms)
mu_monthly, sigma_monthly

(0.04888497148630167, 0.021965962162276254)

In [100]:
monthly_prob_value_add = stats.norm.sf(0, loc=mu_monthly, scale=sigma_monthly)
monthly_prob_value_add

0.9869757311564815

In [99]:
.14950987969774493 / np.sqrt(30) 

0.027296644560112854